In [1]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
import os

# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
video_path = r"E:\MS DA\Semester - 3\Data 255 - 22 - Shim\Assignments\Assignment_4\Source.mp4"
cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
output_folder = r"E:\MS DA\Semester - 3\Data 255 - 22 - Shim\Assignments\Assignment_4\Yolo_output"
os.makedirs(output_folder, exist_ok=True)
out = cv2.VideoWriter(os.path.join(output_folder, 'output.avi'), cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
frame_count = 0
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Save the frame before detection
        cv2.imwrite(os.path.join(output_folder, f'frame_{frame_count}_before.jpg'), frame)

        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Save the frame after detection
        cv2.imwrite(os.path.join(output_folder, f'frame_{frame_count}_after.jpg'), annotated_frame)

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y)))  # x, y center point
            if len(track) > 30:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Write the frame to the output video
        out.write(annotated_frame)

        # Display few sample frames
        if frame_count % 100 == 0:  # Display every 100th frame
            cv2.imshow("Sample Frame", annotated_frame)
            if cv2.waitKey(0) & 0xFF == ord("q"):
                break

        frame_count += 1

    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object, release the output video, and close the display window
cap.release()
out.release()
cv2.destroyAllWindows()


0: 384x640 5 cars, 2 traffic lights, 196.0ms
Speed: 0.0ms preprocess, 196.0ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 2 traffic lights, 86.3ms
Speed: 0.0ms preprocess, 86.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 2 traffic lights, 94.0ms
Speed: 0.0ms preprocess, 94.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 2 traffic lights, 109.4ms
Speed: 0.0ms preprocess, 109.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 2 traffic lights, 155.1ms
Speed: 0.0ms preprocess, 155.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 2 traffic lights, 108.8ms
Speed: 0.0ms preprocess, 108.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 2 traffic lights, 94.2ms
Speed: 0.0ms preprocess, 94.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640

In [11]:
import cv2
import os

# Open the video file
video_path = r"E:\MS DA\Semester - 3\Data 255 - 22 - Shim\Assignments\Assignment_4\Source1.mp4"
cap = cv2.VideoCapture(video_path)

# Define the output folder to save frames
output_folder = r"E:\MS DA\Semester - 3\Data 255 - 22 - Shim\Assignments\Assignment_4\Output"

# Define the frame rate (frames per second) to capture frames
frame_rate = 5  # Consider 5 frames per second

# Define the duration of the video to be considered (in seconds)
video_duration = 5  # Consider 5 seconds of the video

# Initialize variables
frame_count = 0
total_frames = int(cap.get(cv2.CAP_PROP_FPS) * video_duration)

# Loop through the video frames
while cap.isOpened() and frame_count < total_frames:
    ret, frame = cap.read()

    if not ret:
        break

    # Save the frame to the output folder
    cv2.imwrite(os.path.join(output_folder, f'frame_{frame_count}.jpg'), frame)

    # Skip frames based on frame rate
    frames_to_skip = int(cap.get(cv2.CAP_PROP_FPS)) // frame_rate
    for _ in range(frames_to_skip - 1):
        cap.read()

    frame_count += 1

# Release the video capture object
cap.release()

